CNN

In [1]:
import os
import glob
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split

# Base directory where dataset is located
dataset_dir = "/kaggle/input/imagesoasis/Data"

# Automatically detect class folders (instead of hardcoding)
classes = sorted([d for d in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, d))])
print(f"Detected classes: {classes}")

# Load image paths and labels
image_paths, labels = [], []
for class_label, class_name in enumerate(classes):
    class_dir = os.path.join(dataset_dir, class_name)
    files = glob.glob(os.path.join(class_dir, "*.jpg"))
    print(f"Class: {class_name}, Files Found: {len(files)}")
    for file_path in files:
        image_paths.append(file_path)
        labels.append(class_label)

# Check total images loaded
print(f"\nTotal images loaded: {len(image_paths)}")

# Split the dataset
train_paths, test_paths, train_labels, test_labels = train_test_split(
    image_paths, labels, test_size=0.2, stratify=labels, random_state=42
)

# Show split counts
print(f"Training images: {len(train_paths)}")
print(f"Testing images: {len(test_paths)}")


Detected classes: ['Mild Dementia', 'Moderate Dementia', 'Non Demented', 'Very mild Dementia']
Class: Mild Dementia, Files Found: 5002
Class: Moderate Dementia, Files Found: 488
Class: Non Demented, Files Found: 67222
Class: Very mild Dementia, Files Found: 13725

Total images loaded: 86437
Training images: 69149
Testing images: 17288


In [2]:
import tensorflow as tf
from sklearn.preprocessing import LabelBinarizer

# One-hot encode labels
train_paths = np.array(train_paths)
test_paths = np.array(test_paths)

lb = LabelBinarizer()
train_labels = lb.fit_transform(train_labels)
test_labels = lb.transform(test_labels)

IMG_HEIGHT, IMG_WIDTH = 224, 224

def preprocess_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [IMG_HEIGHT, IMG_WIDTH])
    return image / 255.0

def load_dataset(image_paths, labels):
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))
    dataset = dataset.map(lambda x, y: (preprocess_image(x), y))
    return dataset

# Load datasets
train_dataset = load_dataset(train_paths, train_labels).shuffle(1000).batch(32).prefetch(tf.data.AUTOTUNE)
test_dataset = load_dataset(test_paths, test_labels).batch(32).prefetch(tf.data.AUTOTUNE)

# Print dataset objects
print("Train Dataset: 224x224 images with 4 class labels : Mild, Moderate, Non, Very Mild Dementia")
print("Test Dataset: 224x224 images with 4 class labels : Mild, Moderate, Non, Very Mild Dementia")



2025-04-26 00:29:46.186657: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745627386.517848      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745627386.600835      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Train Dataset: 224x224 images with 4 class labels : Mild, Moderate, Non, Very Mild Dementia
Test Dataset: 224x224 images with 4 class labels : Mild, Moderate, Non, Very Mild Dementia


I0000 00:00:1745627401.024659      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1745627401.025386      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


In [3]:
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(classes), activation='softmax')
])

# Print the model summary
print("CNN Model Architecture:")
model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


CNN Model Architecture:


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 222, 222, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 109, 109, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 52, 52, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 26, 26, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 86528)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │      11,075,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 4)                   │             516 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 11,169,476 (42.61 MB)

 Trainable params: 11,169,476 (42.61 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Set up mirrored strategy for multi-GPU
strategy = tf.distribute.MirroredStrategy()
print(f"Number of devices: {strategy.num_replicas_in_sync}")

# Create and compile the model inside the strategy scope
with strategy.scope():
    
    model = Sequential([
        Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 3)),
        MaxPooling2D(),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(4, activation='softmax')  # 4 classes
    ])

    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

# Train the model
EPOCHS = 3
print("Starting training...")
history = model.fit(train_dataset, validation_data=test_dataset, epochs=EPOCHS)


Number of devices: 2
Starting training...
Epoch 1/3


I0000 00:00:1745627409.781323      94 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1745627409.787110      93 cuda_dnn.cc:529] Loaded cuDNN version 90300


2161/2161 ━━━━━━━━━━━━━━━━━━━━ 277s 126ms/step - accuracy: 0.8899 - loss: 0.4667 - val_accuracy: 0.9904 - val_loss: 0.0258
Epoch 2/3
2161/2161 ━━━━━━━━━━━━━━━━━━━━ 241s 111ms/step - accuracy: 0.9932 - loss: 0.0198 - val_accuracy: 0.9943 - val_loss: 0.0174
Epoch 3/3
 626/2161 ━━━━━━━━━━━━━━━━━━━━ 2:42 106ms/step - accuracy: 0.9954 - loss: 0.0114

In [ ]:
# Final evaluation
loss, accuracy = model.evaluate(test_dataset)
print(f"Final Test Accuracy: {accuracy:.4f}")
print(f"Final Test Loss: {loss:.4f}")

# Save model for future use
model.save("/kaggle/working/alzheimers_model.h5")
print("Model saved as 'alzheimers_model.h5' in /kaggle/working/")


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 4))
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.models import load_model

# Load your trained model
model = load_model("/kaggle/working/alzheimers_model.h5")  # or your CNN model path

# Define classes
classes = ["Mild Dementia", "Moderate Dementia", "Non Demented", "Very mild Dementia"]

# Preprocess single image (same as training)
def preprocess_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])
    return image / 255.0

# 🔍 Replace this with your own image path!
test_image_path = "/kaggle/input/imagesoasis/Data/Moderate Dementia/OAS1_0308_MR1_mpr-1_105.jpg"

# Preprocess and predict
image = preprocess_image(test_image_path)
image = tf.expand_dims(image, axis=0)  # Add batch dimension
prediction = model.predict(image)

# Get predicted class
predicted_index = np.argmax(prediction)
predicted_label = classes[predicted_index]

# Output result
print(f"Predicted Class: {predicted_label}")
print(f"Prediction Probabilities: {prediction}")

# Show image
plt.imshow(plt.imread(test_image_path))
plt.title(f"Predicted: {predicted_label}")
plt.axis('off')
plt.show()


In [ ]:
# Extract true labels from test_dataset
true_labels = []
for _, label_batch in test_dataset:
    true_labels.extend(np.argmax(label_batch.numpy(), axis=1))


In [ ]:
# Predict with CNN
cnn_preds = model.predict(test_dataset)
cnn_pred_labels = np.argmax(cnn_preds, axis=1)

# Extract true labels
true_labels = []
for _, labels in test_dataset:
    true_labels.extend(np.argmax(labels.numpy(), axis=1))
true_labels = np.array(true_labels)

# Evaluate CNN
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

cnn_acc = accuracy_score(true_labels, cnn_pred_labels)
cnn_cm = confusion_matrix(true_labels, cnn_pred_labels)
cnn_cr = classification_report(true_labels, cnn_pred_labels, target_names=classes)

print(f"CNN Accuracy: {cnn_acc:.2f}")
print("\nCNN Confusion Matrix:\n", cnn_cm)
print("\nCNN Classification Report:\n", cnn_cr)


In [ ]:
# Save CNN classification report and confusion matrix to CSV
pd.DataFrame(classification_report(true_labels, cnn_pred_labels, output_dict=True)).transpose().to_csv("/content/cnn_report.csv")
pd.DataFrame(cnn_cm).to_csv("/content/cnn_confusion_matrix.csv", index=False)

print("CNN metrics saved to 'cnn_report.csv' and 'cnn_confusion_matrix.csv'")


In [ ]:
# Final evaluation on test data
loss, accuracy = model.evaluate(test_dataset)
print(f"Final Test Accuracy: {accuracy:.4f}")
print(f"Final Test Loss: {loss:.4f}")


In [ ]:
import matplotlib.pyplot as plt

# Plot Accuracy and Loss side-by-side
plt.figure(figsize=(12, 4))

# Accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy During Training')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

# Loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss During Training')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()


In [ ]:
print(model.input_shape)


In [ ]:
import os
import random
import matplotlib.pyplot as plt
import tensorflow as tf

# Set the path to the dataset
data_dir = "/kaggle/input/imagesoasis/Data"  # change if needed
categories = os.listdir(data_dir)

# Plot setup
plt.figure(figsize=(15, 4))

for i, category in enumerate(categories):
    category_path = os.path.join(data_dir, category)
    images = [img for img in os.listdir(category_path) if img.endswith('.jpg')]
    chosen_img = random.choice(images)
    img_path = os.path.join(category_path, chosen_img)

    # Load and display image
    img = tf.keras.utils.load_img(img_path, target_size=(128, 128))
    plt.subplot(1, len(categories), i + 1)
    plt.imshow(img)
    plt.axis('off')
    plt.title(category)

plt.suptitle("Sample MRI Images from Each Class")
plt.tight_layout()
plt.show()


VGG16

In [ ]:
import os
import glob
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
import tensorflow as tf

# Folder path and classes
dataset_dir = "/kaggle/input/imagesoasis/Data"
classes = ["Non Demented", "Very mild Dementia", "Mild Dementia", "Moderate Dementia"]

# Prepare image paths and binary labels (0 = Non Demented, 1 = Demented)
image_paths = []
labels = []

for class_name in classes:
    label = 0 if class_name == "Non Demented" else 1
    class_path = os.path.join(dataset_dir, class_name)
    files = glob.glob(f"{class_path}/*.jpg")
    
    image_paths.extend(files)
    labels.extend([label] * len(files))

# Report counts
print("Total Images Found:", len(image_paths))
print("Class Distribution:")
print(f"  Non Demented (0): {labels.count(0)}")
print(f"  Demented (1): {labels.count(1)}")

# Train-test split
train_paths, test_paths, train_labels, test_labels = train_test_split(
    image_paths, labels, test_size=0.2, stratify=labels, random_state=42
)

print("\n Split:")
print("  Train Samples:", len(train_paths))
print("  Test Samples:", len(test_paths))

# Preprocessing functions
IMG_HEIGHT, IMG_WIDTH = 128, 128

def preprocess_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [IMG_HEIGHT, IMG_WIDTH])
    return image / 255.0

def load_dataset(image_paths, labels):
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))
    dataset = dataset.map(lambda x, y: (preprocess_image(x), y))
    return dataset

# Load datasets
BATCH_SIZE = 32
train_dataset = load_dataset(train_paths, train_labels).shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = load_dataset(test_paths, test_labels).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)



In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
vgg_base.trainable = False  # Freeze VGG16 layers

model = Sequential([
    vgg_base,
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary output
])

model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()


In [ ]:
history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=10
)


In [ ]:
loss, accuracy = model.evaluate(test_dataset)
print(f" Binary Detection Accuracy: {accuracy:.4f}")

# Plot training history
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label="Train Acc")
plt.plot(history.history['val_accuracy'], label="Val Acc")
plt.legend()
plt.title("VGG16 Binary Accuracy")
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Plot Accuracy
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('VGG16 Binary Classification - Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

# Plot Loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('VGG16 Binary Classification - Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
import matplotlib.pyplot as plt

# Load the saved model
model = load_model("/kaggle/working/vgg16_dementia_binary.h5")

# Define class labels (for binary setup)
labels = ["Non Demented", "Demented"]  # 0: Non, 1: Demented


def preprocess_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [128, 128])  # ← changed from 224 to 128
    img = img / 255.0
    return tf.expand_dims(img, axis=0)


# 🔍 Replace this path with any image you want to test
test_image_path = "//kaggle/input/imagesoasis/Data/Moderate Dementia/OAS1_0308_MR1_mpr-1_100.jpg"

# Preprocess and predict
img = preprocess_image(test_image_path)
prediction = model.predict(img)[0][0]

# Determine class
predicted_class = 1 if prediction >= 0.5 else 0
confidence = prediction if predicted_class == 1 else 1 - prediction
label = labels[predicted_class]

# Output result
print(f"Predicted Class: {label}")
print(f"Confidence: {confidence:.2f}")

# Show the image
plt.imshow(plt.imread(test_image_path))
plt.title(f"Predicted: {label} ({confidence:.2f})")
plt.axis('off')
plt.show()
